In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import math
from os import listdir
import data_visualization as dv
import pickle
import scipy 
from scipy.stats import norm
from scipy import stats
from tqdm import tqdm 

In [4]:
PATH = 'D:\Subject_Data\Seth_MatchPennies_Agent_Reaction_Test'
analysis_pull_list = []
with open(PATH+"\\Analysis_Pull_List_React.txt", "r") as pull_file:
    analysis_pull_list = pull_file.read().splitlines()
with open(PATH+"\\Fields_Pull.txt", "r") as fields_pull:
    fields_pull = fields_pull.read().splitlines()
    
num_trials = 50
num_blocks = 2
tot_trials = num_trials*num_blocks

num_subjects = 1

## TARGET POSITION VALUES
# Target information for Right Hand
start1x = 0.55
start1y = 0.15
# Give an extra 2.5cm so when people drift we don't count that as an indecision
start1_diameter = 0.015 + 0.025
target1x = 0.6611
target1y = 0.31664
t1_pos = np.sqrt(target1x**2 + target1y**2)
target1_diameter = 0.08

target2x = 2*start1x - target1x
target2y = 0.31664
t2_pos = np.sqrt(target2x**2 + target2y**2)
target2_diameter = 0.08

# for Left Hand
start2x = -0.55
start2y = 0.15
start2_diameter = 0.015

target3x = target1x - 2*start1x
target3y = 0.31664
t3_pos = np.sqrt(target3x**2 + target3y**2)
target3_diameter = 0.08

target4x = 2*start2x - target3x
target4y = 0.3
t4_pos = np.sqrt(target4x**2 + target4y**2)
target4_diameter = 0.08

# Timing target
timing_targetx = start2x
timing_targety = 0.31664
timing_target_pos = np.sqrt(timing_targetx**2 + timing_targety**2)
timing_target_diameter = 0.08


In [ ]:
#%% Get reaction time data
reaction_timing_trial_time = 2500
rt_decision_array = np.empty((num_subjects,num_trials,num_blocks))
rmt_decision_array = np.empty((num_subjects, num_trials,num_blocks))
rt_trial_start = np.zeros((num_subjects, num_trials,num_blocks))*np.nan
reaction_movement_time = np.zeros([num_subjects, num_trials,num_blocks])*np.nan
reaction_time = np.zeros([num_subjects, num_trials,num_blocks])*np.nan
rt_hand_pos_data = np.zeros((num_subjects, num_trials,num_blocks, reaction_timing_trial_time, 4))*np.nan

timing_reach_time = np.zeros([num_subjects, num_trials,num_blocks])*np.nan
for w in range(num_subjects):
    sub_name = analysis_pull_list[w]
    print(w, sub_name)
    block_number = 1
    tp_num = 1
    sub_name = analysis_pull_list[w]
    path1 = PATH+'\\'+sub_name
    file = path1+f'\\{sub_name}Trial_Table.csv'
    reaction_trial_table = pd.read_csv(file)
    #reaction_trial_table = reaction_trial_table.loc[reaction_trial_table['TP_Row']==1]
    for i in tqdm((range(num_trials)):
            block_number = reaction_trial_table.iloc[i]['Block_Row']
            tp_num = reaction_trial_table.iloc[i]['TP_Row']
            block_trial_num = reaction_trial_table.iloc[i]['Block_Step']
            x = tp_num - 1
            c = block_trial_num - 1
            filename = PATH+f"\\{sub_name}\\Seth_MatchPennies_Agent_Pilot_v3_{sub_name}_C{block_number}_TP{tp_num}_T{block_trial_num}.csv"

            data = pd.read_csv(filename, low_memory=False)
            rt_trial_start[w,c]= int(data[data['Event_Codes']=='E_SOUND_SIGNAL'].index[0]) #if event codes column is equal to the list event codes, give the index (time)
            rt_trial_start = rt_trial_start.astype(int)
            end_time = rt_trial_start[w,c]+reaction_timing_trial_time

            data = np.array(data)
            hand_data = data[:,0:4] # data is only columns 0 thru 3 (RHX,RHY,LHX,LHY)
            hand_data = hand_data.astype(float)
            rt_hand_pos_data[w,c,:,:] = hand_data[rt_trial_start[w,c]:end_time,:]
            # LEFT HAND
            lhx = rt_hand_pos_data[w,c,:,2]
            lhy = rt_hand_pos_data[w,c,:,3]
            if tp_num == 1:
                q = np.argwhere(np.sqrt((lhx-target3x)**2 + (lhy-target3y)**2) < target3_diameter/2)
                r = np.argwhere(np.sqrt((lhx-target4x)**2 + (lhy-target4y)**2) < target4_diameter/2) 
                s = np.argwhere(np.sqrt((lhx-start2x)**2 + (lhy-start2y)**2) > start2_diameter/2)
                if np.size(s)>0: 
                    reaction_time[w,c] = s[0]
                if np.size(q)>0: #if LH enters right target
                    rmt_decision_array[w,c] = 1
                    reaction_movement_time[w,c] = q[0]
                elif np.size(r)>0: #if LH enters left target
                    rmt_decision_array[w,c] = -1   
                    reaction_movement_time[w,c] = r[0]
            # Used if doing timing trials
            if tp_num == 2:
                q = np.argwhere(np.sqrt((lhx-timing_targetx)**2 + (lhy-timing_targety)**2) < timing_target_diameter/2) # THIS NEEDS TO BE THE CENTER TARGET... x poisition should be the start
                if np.size(q)>0:
                    timing_reach_time[w,c] = q[0]
`